https://www.kaggle.com/code/cpmpml/ultra-fast-distance-matrix-computation/notebook

Distance matrix between atoms of a single molecule are used in many public kernels to compute features, for instance Coulomb interaction, Van de Walls interaction, and Yukawa interactions, 

This kernel shows how to speed up the distance matrix computation tremendously compared to the already fast version used in [coulomb_interaction - speed up!](https://www.kaggle.com/rio114/coulomb-interaction-speed-up).  The code from that kernel takes 2 minutes for all molecules.  

Here we provide a code that runs in 3 seconds, i.e. 40 times faster.

This speedup is nice, but the code optimization technique used in this kernel is rather generic and can be reused in other context.

V4 update.  @jmtest has suggested a nice improvement using einssum.  I added his version.  This brings down time to about 1.2 second.  We can do even better using numba, which brings down time to about 0.4 second. This is more than 250 faster than original code.

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input"))

# Any results you write to the current directory are saved as output.

['.csv', 'dipole_moments.csv', 'magnetic_shielding_tensors.csv', 'mulliken_charges.csv', 'potential_energy.csv', 'sample_submission.csv', 'scalar_coupling_contributions.csv', 'structures.csv', 'test.csv', 'train.csv']


In [6]:
structures = pd.read_csv('input/structures.csv')

structures

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
...,...,...,...,...,...,...
2358870,dsgdb9nsd_133885,11,H,-1.454004,-0.967309,1.459246
2358871,dsgdb9nsd_133885,12,H,0.277779,-2.697872,0.195770
2358872,dsgdb9nsd_133885,13,H,2.515854,-1.151784,0.527369
2358873,dsgdb9nsd_133885,14,H,0.013699,1.199431,-1.680192


> An efficient way of computing distance matrix is provided in the [coulomb_interaction - speed up!](https://www.kaggle.com/rio114/coulomb-interaction-speed-up) kernel from Ryoji Nomura.  Please drop me a comment if the code is from another author and I'll correct the attribution.

In [7]:
structures_idx = structures.set_index('molecule_name')

def get_dist_matrix(df_structures_idx, molecule):
    df_temp = df_structures_idx.loc[molecule]
    locs = df_temp[['x','y','z']].values
    num_atoms = len(locs)
    loc_tile = np.tile(locs.T, (num_atoms,1,1))
    dist_mat = np.sqrt((loc_tile - loc_tile.T)**2).sum(axis=1)
    return dist_mat

Let's see how much time this takes for all molecules

In [8]:
for molecule in tqdm_notebook(structures.molecule_name.unique()):
    get_dist_matrix(structures_idx, molecule)

/tmp/ipykernel_4616/1472194569.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for molecule in tqdm_notebook(structures.molecule_name.unique()):


  0%|          | 0/130789 [00:00<?, ?it/s]

This takes 2 minutes.

Can we do better?

A way to accelerate code is to replace pandas operations by numpy operations.  The code above already does this in a very clever way by using the numpy tile function and vectorized distance computation.

Still, it creates a pandas dataframe for each molecule, then converts it into numpy array.  This is slow.  Also, filtering by the molecule name to extract the structure information for a given molecule is expensive.  We can speed up both by using numpy arrays up front.  Let's create the array we need.

In [9]:
xyz = structures[['x','y','z']].values


This will remove the need to first create a data frame then convert it into a numpy array.  This is nice, but we need a way to extract the part relevant to a given molecule from this single numpy array.  

The original structures data frame is sorted by molecule, meaning that information for a given molecule is in a consecutive set of rows. We can then precompute the indices of each molecule segment.  

We first compute the number of rows for each molecule, then using cumsum to get indices for molecule changes.  We then store this as a numpy array where we add 0 as the first value.  The molecule names can be retrieved as the index of the series.

In [10]:
ss = structures.groupby('molecule_name').size()
ss = ss.cumsum()
ss

molecule_name
dsgdb9nsd_000001          5
dsgdb9nsd_000002          9
dsgdb9nsd_000003         12
dsgdb9nsd_000004         16
dsgdb9nsd_000005         19
                     ...   
dsgdb9nsd_133881    2358808
dsgdb9nsd_133882    2358824
dsgdb9nsd_133883    2358841
dsgdb9nsd_133884    2358859
dsgdb9nsd_133885    2358875
Length: 130789, dtype: int64

In [11]:
ssx = np.zeros(len(ss) + 1, 'int')
ssx[1:] = ss
ssx

array([      0,       5,       9, ..., 2358841, 2358859, 2358875])

Then, to retrieve information for the i-th molecule, we slice the xyz array using the indices we just compuited.  For instance, to get the information for the 20th molecule we do:

In [12]:
molecule_id = 20
print(ss.index[molecule_id])
start_molecule = ssx[molecule_id]
end_molecule = ssx[molecule_id+1]
xyz[start_molecule:end_molecule]

dsgdb9nsd_000021


array([[-3.21588811e-02,  1.54021598e+00,  1.07445588e-02],
       [ 3.38174132e-02,  7.45852170e-03,  1.80716120e-03],
       [ 7.13755719e-01, -5.08564040e-01, -1.27296949e+00],
       [ 7.38489986e-01, -5.22208864e-01,  1.25750907e+00],
       [ 9.75227360e-01,  1.97414341e+00,  5.28361840e-03],
       [-5.61967693e-01,  1.92232326e+00, -8.68354181e-01],
       [-5.48172394e-01,  1.91268414e+00,  9.02080385e-01],
       [-9.97525248e-01, -3.73105784e-01,  9.77296510e-03],
       [ 1.75070021e+00, -1.55867844e-01, -1.33269631e+00],
       [ 7.34466914e-01, -1.60328288e+00, -1.29939696e+00],
       [ 1.93670105e-01, -1.60855893e-01, -2.17195508e+00],
       [ 2.36119903e-01, -1.83820171e-01,  2.17005260e+00],
       [ 7.59640833e-01, -1.61713153e+00,  1.27226101e+00],
       [ 1.77635866e+00, -1.69853184e-01,  1.30053019e+00]])

We can compare with the information we get from the original pandas dataframe

In [13]:
structures_idx.loc['dsgdb9nsd_000022'][['x', 'y', 'z']].values

array([[-0.03315871,  1.54782566, -0.00438815],
       [-0.01108534,  0.01859095,  0.01676117],
       [ 0.70938384, -0.53596711,  1.23978444],
       [-1.33219841, -0.51712554,  0.05236057],
       [ 0.98215692,  1.957159  , -0.03182225],
       [-0.563881  ,  1.92087623, -0.88845865],
       [-0.54082548,  1.93340945,  0.88575489],
       [ 0.51042122, -0.33767241, -0.88819176],
       [ 0.22240492, -0.1878586 ,  2.15650245],
       [ 0.68242024, -1.62887638,  1.23305141],
       [ 1.75434301, -0.2124695 ,  1.25524188],
       [-1.81818681, -0.15577435, -0.69553962]])

Looks good.

We can now rewrite our function using our arrays.


In [14]:
def get_fast_dist_matrix(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]    
    num_atoms = end_molecule - start_molecule
    loc_tile = np.tile(locs.T, (num_atoms,1,1))
    dist_mat = np.sqrt((loc_tile - loc_tile.T)**2).sum(axis=1)
    return dist_mat

Let's check we get the same result with both techniques.  We use the smallest molecule for the sake of simplicity. It is water.

In [15]:
molecule_id = 2
molecule = ss.index[molecule_id]
print(molecule)
get_fast_dist_matrix(xyz, ssx, molecule_id)

dsgdb9nsd_000003


array([[0.        , 1.06216132, 1.23631216],
       [1.06216132, 0.        , 2.08808559],
       [1.23631216, 2.08808559, 0.        ]])

In [16]:
get_dist_matrix(structures_idx, molecule)

array([[0.        , 1.06216132, 1.23631216],
       [1.06216132, 0.        , 2.08808559],
       [1.23631216, 2.08808559, 0.        ]])

Looks good.

We can now benchmark our version.

In [17]:
for molecule_id in tqdm_notebook(range(structures.molecule_name.nunique())):
    get_fast_dist_matrix(xyz, ssx, molecule_id)

/tmp/ipykernel_4616/1796620152.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for molecule_id in tqdm_notebook(range(structures.molecule_name.nunique())):


  0%|          | 0/130789 [00:00<?, ?it/s]

We are down to 3 seconds from 2 minutes!

A nice improvement was proposed by @jmtest in the comments.  In order to benchmark our code and his, we need to use a more precise way than tqdm.  Let's use the `%time` magic. It requires us to wrap the code in a function

In [18]:
def ultra_fast_dist_matrices(xyz, ssx):
    for molecule_id in range(structures.molecule_name.nunique()):
        get_fast_dist_matrix(xyz, ssx, molecule_id)

In [19]:
%time ultra_fast_dist_matrices(xyz, ssx)

CPU times: user 2.44 s, sys: 8.49 ms, total: 2.45 s
Wall time: 2.45 s


Let's do the same for @jmtest code

In [20]:
def sofast_dist(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]     
    d=locs[:,None,:]-locs
    return np.sqrt(np.einsum('ijk,ijk->ij',d,d))

def sofast_dist_matrices(xyz, ssx):
    for molecule_id in range(structures.molecule_name.nunique()):
        sofast_dist(xyz, ssx, molecule_id)

In [21]:
%time sofast_dist_matrices(xyz, ssx)

CPU times: user 1.2 s, sys: 9.54 ms, total: 1.21 s
Wall time: 1.21 s


almost a 3x speedup.  

Can we do better?  The ultimate weapon to accelerate vectorized operaitons in Python is to use a compiler.  Numba is my favorite because we stay in Python.  Alternatives are Cython compiler, or calling C/C++ code.  In my experience Numba provides almost the same speed as compiled C code if there are no indirect array access, which is the case here.  

Numba works by annotating code.

In [24]:
from numba import jit
from math import sqrt

@jit
def numba_dist_matrix(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]     
   # return locs
    num_atoms = end_molecule - start_molecule
    dmat = np.zeros((num_atoms, num_atoms))
    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            d = sqrt((locs[i,0] - locs[j,0])**2 + (locs[i,1] - locs[j,1])**2 + (locs[i,2] - locs[j,2])**2)
            dmat[i,j] = d
            dmat[j,i] = d
    return dmat

def numba_dist_matrices(xyz, ssx):
    for molecule_id in range(structures.molecule_name.nunique()):
        numba_dist_matrix(xyz, ssx, molecule_id)

SystemError: initialization of _internal failed without raising an exception

Let's make sure our code is correct by comparing with the previous one.

In [ ]:
molecule_id = 2
molecule = ss.index[molecule_id]
print(molecule)
numba_dist_matrix(xyz, ssx, molecule_id)

dsgdb9nsd_000003


array([[0.        , 0.96210681, 0.96210681],
       [0.96210681, 0.        , 1.51335787],
       [0.96210681, 1.51335787, 0.        ]])

In [ ]:
sofast_dist(xyz, ssx, molecule_id)

array([[0.        , 0.96210681, 0.96210681],
       [0.96210681, 0.        , 1.51335787],
       [0.96210681, 1.51335787, 0.        ]])

Looks good

In [ ]:
%time numba_dist_matrices(xyz, ssx)

CPU times: user 384 ms, sys: 20 ms, total: 404 ms
Wall time: 404 ms


A further 3x speedup!  We are now about 250 times faster than the original code...


It was suggested in comments that scipy distance would be faster.  Let's have a look.  scipy requires two functions to output a square matrix.

In [ ]:
from scipy.spatial.distance import pdist, squareform

def scipy_dist_matrix(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]     
    cmat = pdist(locs)
    dmat = squareform(cmat, force='tomatrix')
    return dmat

Let's check that we get the same result.

In [ ]:
scipy_dist_matrix(xyz, ssx, molecule_id)

array([[0.        , 0.96210681, 0.96210681],
       [0.96210681, 0.        , 1.51335787],
       [0.96210681, 1.51335787, 0.        ]])

We can now benchmark.

In [ ]:
def scipy_dist_matrices(xyz, ssx):
    for molecule_id in range(structures.molecule_name.nunique()):
        scipy_dist_matrix(xyz, ssx, molecule_id)

In [ ]:
%time scipy_dist_matrices(xyz, ssx)

CPU times: user 4 s, sys: 32 ms, total: 4.04 s
Wall time: 4.03 s


Results are disappointing.

If this kernel gets enough votes then I'll share another kernel with ultra fast way to compute angles and dighedral angles with numpy ;)

@ilyivanchenko points out that there is an issue in the get_dist_matrix thagt I borrowed from [coulomb_interaction - speed up!](https://www.kaggle.com/rio114/coulomb-interaction-speed-up).

In [ ]:
epsilon = 1e-5

def get_dist_matrix_assert(df_structures_idx, molecule):
    df_temp = df_structures_idx.loc[molecule]
    locs = df_temp[['x','y','z']].values
    num_atoms = len(locs)
    loc_tile = np.tile(locs.T, (num_atoms,1,1))
    dist_mat = np.sqrt((loc_tile - loc_tile.T)**2).sum(axis=1)
    assert np.abs(dist_mat[0,1] - np.linalg.norm(locs[0] - locs[1])) < epsilon
    return dist_mat

for molecule in tqdm_notebook(structures.molecule_name.unique()[660:]):
    try:
        get_dist_matrix_assert(structures_idx, molecule)
    except: 
        print('assertion error on', molecule)
        break
        



assertion error on dsgdb9nsd_000676



Let's see if we have the issue in our code.

In [ ]:
@jit
def numba_dist_matrix_ssert(xyz, ssx, molecule_id):
    start_molecule, end_molecule = ssx[molecule_id], ssx[molecule_id+1]
    locs = xyz[start_molecule:end_molecule]     
   # return locs
    num_atoms = end_molecule - start_molecule
    dmat = np.zeros((num_atoms, num_atoms))
    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            d = sqrt((locs[i,0] - locs[j,0])**2 + (locs[i,1] - locs[j,1])**2 + (locs[i,2] - locs[j,2])**2)
            dmat[i,j] = d
            dmat[j,i] = d
    assert np.abs(dmat[0,1] - np.linalg.norm(locs[0] - locs[1])) < epsilon
    return dmat

for molecule_id in range(structures.molecule_name.nunique()):
    numba_dist_matrix_ssert(xyz, ssx, molecule_id)

The code runs fine.